In [1]:
from SyncMap_Draft.core import SymmetricalSyncMap, LightSyncMap, NodeSyncMap

ImportError: cannot import name 'NodeSyncMap' from 'SyncMap_Draft.core' (/home/thunderbird/SyncMap-Draft/SyncMap_Draft/core.py)